In [1]:
# hide ssl warnings for this example.
import requests
requests.packages.urllib3.disable_warnings()

# Finding data with python-fmrest

This is a short example on finding records vis python-fmrest.

In [2]:
import fmrest

## Login

In [3]:
fms = fmrest.Server('https://10.211.55.15',
                    user='admin',
                    password='admin',
                    database='Contacts',
                    layout='Contacts',
                    verify_ssl=False
                   )
fms.login()

'6fef7062d548599481d23f3868491106efaab4f85b5071f5c655'

## Specify find queries and retrieve foundset and record

We want to find records in Contacts where the name field matches 'John Doe'.

In [4]:
find_query = [{'name': 'John Doe'}]
foundset = fms.find(find_query)
foundset

<Foundset fetched_records=0 is_complete=False>

We have our foundset and can iterate through it. fetched_records=0 means that we haven't consumed the Foundset yet.

In [5]:
for record in foundset:
    print(record)

<Record id=44 modification_id=1>


In [6]:
foundset

<Foundset fetched_records=1 is_complete=True>

Looks like Record 44 is our only record in the foundset. Let's look at it:

In [7]:
record = foundset[0]
print(record.keys()) # all the field names on the layout, including portals
print(record.values()) # all the value corresponding to the field names

['id', 'name', 'drink', 'portrait', 'creation', 'modification', 'g_note_id_active', 'Notes_active::note', 'date', 'recordId', 'modId', 'portal_notes']
['44', 'John Doe', 'Pepsi', 'https://10.211.55.15/Streaming_SSL/MainDB/540D3DFD108F6C9136507F43229EEE2B06B5E884EFF03BC632CF2D03D704DDEA.jpg?RCType=EmbeddedRCFileProcessor', '11/05/2017 20:14:18', '11/05/2017 20:15:58', '', '', '', '44', '1', <Foundset fetched_records=0 is_complete=False>]


Above we can see all available keys and values. If we want to acces a value, we can use the Record's attributes:

In [8]:
record.name

'John Doe'

In [9]:
record.drink

'Pepsi'

## Using wildcards

The FileMaker Data API supports the same operators FileMaker Pro does. So let's go ahead and broaden our search.

In [10]:
find_query = [{'name': 'John*'}]
foundset = fms.find(find_query)

for record in foundset:
    print(record.id, record.name)

44 John Doe
45 John Smith


Using the wildcard we match all the Johns.

## Multiple find requests

Multiple find requests are also supported. Again, just like in FileMaker Pro.

Let's find all Johns and Joes, but not John Does.

In [11]:
find_query = [
    {'name': 'John'},
    {'name': 'Joe'},
    {'name': 'John Doe', 'omit': 'true'}
]
foundset = fms.find(find_query)

for record in foundset:
    print(record.name)

Joe Example
John Smith


## Sorting the result

You can control the order of the results by specifying the sort parameter.

In [12]:
order_by = [{'fieldName': 'name', 'sortOrder': 'descend'}] #descending
foundset = fms.find(find_query, sort=order_by)

for record in foundset:
    print(record.name)

print('---')

order_by = [{'fieldName': 'name', 'sortOrder': 'ascend'}] #ascending
foundset = fms.find(find_query, sort=order_by)

for record in foundset:
    print(record.name)

John Smith
Joe Example
---
Joe Example
John Smith


## Limiting the result

To create more efficient requests, you can limit the data being returned.

#### Limit with offset

Only return from the second record.

In [13]:
foundset = fms.find(find_query, sort=order_by, offset=2)
for record in foundset:
    print(record.name)

John Smith


#### Limit with range

Only return one record.

In [14]:
foundset = fms.find(find_query, sort=order_by, range_=1)
foundset[0].name

'Joe Example'

Of course, you can combine these parameters as you like. Defaults are offset 1 (which is the first record), range 100.

#### Limit data returned by portals

By specifying portals, you can prevent certain portal data to be returned, even if portals are present on your layout.

In [15]:
portals = [{'name':'notes', 'offset':1, 'range': 2}]
foundset = fms.find(find_query, portals=portals)

for row in foundset[0].portal_notes:
    print(row['Notes::note'])

A note for Joe.
Another note for Joe.


The name is the object name as specified in the Inspector in FileMaker Pro. Offset and range are optional with default values 1 and 50.

Not specifying the portals parameter will return all portals on the layout with the default offset and range.